In [1]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta, date
from collections import defaultdict
import json

In [2]:
df = pd.read_csv('./data/race_data/2019.csv')

In [3]:
new_times_lap = []
for index, time in df['Lap Time'].iteritems():
    if time.count(':') == 1:
        new_times_lap.append(datetime.strptime(time, '%M:%S.%f'))
    else:
        new_times_lap.append(datetime.strptime(time, '%H:%M:%S.%f'))
df['Lap Time'] = new_times_lap

In [4]:
new_times_s = []
date_index = 100000
for index, time in df['Session Time'].iteritems():
    if time.startswith('24') and time.count(':') > 1:
        date_index = index
        new_time = '01:00' + time[2:]
        new_date = datetime.strptime(new_time, '%d:%H:%M:%S.%f')
        new_date_plus = new_date + timedelta(days=1)
        new_times_s.append(new_date_plus)
    elif index > date_index:
        new_time = '01:00:0' + time
        new_date = datetime.strptime(new_time, '%d:%H:%M:%S.%f')
        new_date_plus = new_date + timedelta(days=1)
        new_times_s.append(new_date_plus)
    elif time.count(':') == 1:
        new_times_s.append(datetime.strptime(time, '%M:%S.%f'))
    else:
        new_times_s.append(datetime.strptime(time, '%H:%M:%S.%f'))
df['Session Time'] = new_times_s

In [5]:
df['Prev Session Time'] = df['Session Time'].subtract(df['Lap Time'])

In [8]:
for i in range(1,14):
    num_str = str(i)
    if i < 10: num_str = '0' + num_str
    num_str = 'S' + num_str
    
    vals = []
    for ind, split_time in df[num_str].iteritems():
        vals.append(timedelta(seconds=split_time))
    df[num_str] = vals
    totals = df[num_str] + df['Prev Session Time']
    
    for j in range(i-1, 0, -1):
        new_num_str = str(j)
        if j < 10: new_num_str = '0' + new_num_str
        new_num_str = 'S' + new_num_str
        totals += df[new_num_str]
    df[num_str+'totals'] = totals

TypeError: unsupported type for timedelta seconds component: Timedelta

In [9]:
df.to_csv('./data/2019_mod.csv', index=False, date_format='%Y-%m-%d %H:%M:%S.%f')

In [8]:
section_mod = defaultdict(list)
for index, row in df.iterrows():
    section_mod[row['Driver']].append((1, row['Prev Session Time'].total_seconds(), row['S01totals'].total_seconds(), row.Car))
    section_mod[row['Driver']].append((2, row['S01totals'].total_seconds(), row['S02totals'].total_seconds(), row.Car))
    section_mod[row['Driver']].append((3, row['S02totals'].total_seconds(), row['S03totals'].total_seconds(), row.Car))
    section_mod[row['Driver']].append((4, row['S03totals'].total_seconds(), row['S04totals'].total_seconds(), row.Car))
    section_mod[row['Driver']].append((5, row['S04totals'].total_seconds(), row['S05totals'].total_seconds(), row.Car))
    section_mod[row['Driver']].append((6, row['S05totals'].total_seconds(), row['S06totals'].total_seconds(), row.Car))
    section_mod[row['Driver']].append((7, row['S06totals'].total_seconds(), row['S07totals'].total_seconds(), row.Car))
    section_mod[row['Driver']].append((8, row['S07totals'].total_seconds(), row['S08totals'].total_seconds(), row.Car))
    section_mod[row['Driver']].append((9, row['S08totals'].total_seconds(), row['S09totals'].total_seconds(), row.Car))
    section_mod[row['Driver']].append((10, row['S09totals'].total_seconds(), row['S10totals'].total_seconds(), row.Car))
    section_mod[row['Driver']].append((11, row['S10totals'].total_seconds(), row['S11totals'].total_seconds(), row.Car))
    section_mod[row['Driver']].append((12, row['S11totals'].total_seconds(), row['S12totals'].total_seconds(), row.Car))
    section_mod[row['Driver']].append((13, row['S12totals'].total_seconds(), row['S13totals'].total_seconds(), row.Car))

In [9]:
MAXTIME = df['S13totals'].max().total_seconds()
def roundTo(x, base=20):
    return base * round(x/base)
MAXTIME = roundTo(MAXTIME, 20)

In [10]:
pit_times = {}
for car in list(pd.unique(df.Car)):
    df_car = df.loc[(df['Car'] == car) & (df['Location'] == 'Pit')]
    pt = list(pd.unique(df_car['Session Time']))
    d_zero = np.datetime64('1900-01-01T00:00:00')
    pt = [(d - d_zero) / np.timedelta64(1, 's') for d in pt]
    pt = [0] + pt
    pit_times[car] = pt

In [36]:
# last_pit = {}
# for t in range(20, MAXTIME, 20):
#     all_times = []
#     for car in pit_times.keys():
#         lp = max(filter(lambda pt: pt < t, pit_times[car]))
#         all_times.append(t - lp)
#     last_pit[t] = np.average(all_times) / 60

In [11]:
distances = {1: 1876, 2: 1751.33, 3: 1768.67, 4: 2352, 5: 1488, 6: 1475, 7: 1427, 
             8: 1428, 9: 1457, 10: 1484, 11: 1283, 12: 120, 13: 886.83}

num_drivers = defaultdict(lambda: defaultdict(list))
concentration = defaultdict(lambda: defaultdict(float))
last_pit = {}
for t in range(20, MAXTIME, 20):
    cars = []
    for driver in section_mod:
        for interval in section_mod[driver]:
            if interval[1] <= t <= interval[2]:
                num_drivers[t][interval[0]].append((interval[1], interval[2]))
                cars.append(interval[3])
                break
    for section in num_drivers[t]:
        concentration[t][section] = (len(num_drivers[t][section]) / distances[section]) * 1000
        # cars per 1000 feet
    
    all_times = []
    for car in cars:
        lp = max(filter(lambda pt: pt < t, pit_times[car]))
        all_times.append(t - lp)
    last_pit[t] = np.max(all_times) / 60

In [12]:
with open('20_count_wspeed.json', 'w') as outfile:
    json.dump(num_drivers, outfile)
with open('./data/concentrations.csv', 'w') as f:
    f.write('Time,Sector,Concentration\n')
    for time in concentration.keys():
        for sector in range(1,14):
            f.write("%s,%s,%s\n"%(time, sector, concentration[time].get(sector, 0)))

In [13]:
with open('./data/avg_time_since_pit.csv', 'w') as f:
    f.write('Time,Last Pit\n')
    for time, lp in last_pit.items():
        f.write("%s,%s\n"%(time, lp))